In [1]:
import pandas as pd
import pickle
import numpy as np

# Data Reader

In [2]:
data = pd.read_csv('./datasets/trainset.csv',delimiter=" ")

In [3]:
data_v = pd.read_csv('./datasets/track1_testset.csv',' ')

In [4]:
def data_reader(data):
    tmp_list = data['user_click_history'].tolist()
    time_list = data['time'].tolist()
    return tmp_list,time_list

# Events

In [5]:
#time_step=600
def item_time_window(tmp_list):
    #input:raw data
    #output:last click item time,each user's item list in time window,item list 2D
    
    item_event,last_time,flat_event = [],[],[]
    for user_item in tmp_list:
        item_list = user_item.split(',')
        user_item_seq,item_seq,item_seq_str =[],[],[]
        time_tmp = 0
        for item in item_list:
            item_time = item.split(':')
            if not item_seq or int(item_time[1])-time_tmp<600:
                time_tmp = int(item_time[1])
                item_seq.append(int(item_time[0]))
                item_seq_str.append(str(item_time[0]))
            else:
                user_item_seq.append(item_seq)
                flat_event.append(item_seq_str)
                item_seq = []
                item_seq_str = []
                item_seq.append(int(item_time[0]))
                item_seq_str.append(str(item_time[0]))
                time_tmp = int(item_time[1])
        user_item_seq.append(item_seq)
        flat_event.append(item_seq_str)
        item_event.append(user_item_seq)
        last_time.append(time_tmp)
    return last_time,item_event,flat_event

In [6]:
def time_interval_weights(tmp_list,time_list,bins):
    #input:raw data
    #output:time between last_time and raw time
    
    users_time = []
    for n,user in enumerate(tmp_list):
        user_time = []
        item_list = user.split(',')
        pur_time = int(time_list[n])
        for item in item_list:
            time = int(item.split(':')[1])
            user_time.append(pur_time-time)
        users_time.append(user_time)
    time_weights = [0]*len(time_list)
    for n,aa in enumerate(users_time):
        aa = np.array(aa).astype('float64')
        time_weights[n]=pd.cut(aa-aa[-1]+0.1,bins,labels =[float('%.2f'%(1-0.05*i)) for i in range(9)]).tolist()
    return time_weights

In [7]:
def data_item_event(tmp_list):
    users_item = []
    for user_item in tmp_list:
        item_list = user_item.split(',')
        user_item =[]
        for item in item_list:
            item_time = item.split(':')
            user_item.append(int(item_time[0]))
        users_item.append(user_item)
    return users_item

In [8]:
def preprocessed(data,data_name,flat_name,bins):
    tmp_list,time_list = data_reader(data)
    last_time,item_event,flat_event = item_time_window(tmp_list)
    time_weights = time_interval_weights(tmp_list,time_list,bins)
    users_item = data_item_event(tmp_list)
    data.loc[:,'item_event'] = users_item
    data.loc[:,'last_time']=last_time
    aaa = (data['time']-data['last_time'])
    interval = pd.cut(aaa,bins,labels =[1-0.05*i for i in range(9)])
    data.loc[:,'time_weights']=time_weights
    data.loc[:,'interval']=interval
    new_data = data[['user_id','item_event','user_protrait','exposed_items','labels','interval','time_weights']]
    new_data.to_csv('./datasets/'+data_name+'.csv',index=False)
    pickle.dump(flat_event, open('./datasets/'+flat_name+'.txt','wb'),0)

In [9]:
#cut by day
bins = [0,86400,172800,259200,345600,432000,518400,604800,1296000,10000000000]

In [10]:
preprocessed(data,'event','flat_events',bins)

/home/liwenliang/anaconda3/envs/rob/lib/python3.6/site-packages/pandas/core/dtypes/missing.py:495: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val = np.array(val, copy=False)
/home/liwenliang/anaconda3/envs/rob/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [11]:
preprocessed(data_v,'event_v','flat_events_v',bins)